# アソシエーション分析をやってみた
- mlxtend モジュール使いやすい
- 今回は以下２つを使用  
  1. 「support（出現確率）」
  1. 「lift（同時出現の強さ）」


In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

INPUT_DIR = "../input"
OUTPUT_DIR = "../reference"

pd.set_option("display.max_columns", 800)
pd.set_option("display.max_rows", 300)

In [6]:
product_master_df = pd.read_csv(os.path.join(INPUT_DIR, 'product_master.csv'), dtype={ 'JAN': str })

log_df = pd.read_csv(os.path.join(INPUT_DIR, 'carlog.csv'), dtype={ 'value_1': str }, date_parser=['date'])

test_df = pd.read_csv(os.path.join(INPUT_DIR, 'test.csv'))

trainデータに限定する

In [7]:
test_sessions = test_df["session_id"].unique()
idx_test = log_df["session_id"].isin(test_sessions)
log_df = log_df[~idx_test].reset_index(drop=True)

In [8]:
log_df.shape

(14299716, 18)

In [10]:
TARGET_CATEGORIES = [
    # お酒に関するもの
    'ビール系__RTD', 'ビール系__ビール系', 'ビール系__ノンアルコール',

    # お菓子に関するもの
    'スナック・キャンディー__スナック', 
    'チョコ・ビスクラ__チョコレート', 
    'スナック・キャンディー__ガム', 
    'スナック・キャンディー__シリアル',
    'アイスクリーム__ノベルティー', 
    '和菓子__米菓',

    # 飲料に関するもの
    '水・炭酸水__大型PET（炭酸水）',
    '水・炭酸水__小型PET（炭酸水）',
    '缶飲料__コーヒー（缶）',
    '小型PET__コーヒー（小型PET）',
    '大型PET__無糖茶（大型PET）',

    # 麺類
    '麺類__カップ麺',
]

In [21]:
cat2id = dict(zip(product_master_df["category_name"], product_master_df["category_id"]))

id2cat = dict(zip(product_master_df["category_id"], product_master_df["category_name"]))

In [14]:
TARGET_IDS = pd.Series(TARGET_CATEGORIES).map(cat2id).values.tolist()
category_id2code = dict(zip(TARGET_IDS, TARGET_CATEGORIES))

In [15]:
def only_payment_session_record(input_log_df):
    """支払いが紐づくセッションへ絞り込みを行なう"""
    payed_sessions = input_log_df[input_log_df['is_payment'] == 1]['session_id'].unique()
    idx = input_log_df['session_id'].isin(payed_sessions)
    out_df = input_log_df[idx].reset_index(drop=True)
    return out_df


def only_purchase_records(input_df: pd.DataFrame) -> pd.DataFrame:
    idx = input_df['kind_1'] == '商品'
    out_df = input_df[idx].reset_index(drop=True)
    return out_df


def create_payment(input_df: pd.DataFrame) -> pd.DataFrame:
    """
    ログデータから session_id / JAN ごとの購買情報に変換します.

    Args:
        input_df:
            レジカートログデータ

    Returns:
        session_id, JAN, n_items (合計購買数) の DataFrame
    """

    # 購買情報は商品のものだけ.
    out_df = only_purchase_records(input_df)
    out_df = out_df.groupby(['session_id', 'value_1'])['n_items'].sum().reset_index()
    out_df = out_df.rename(columns={
        'value_1': 'JAN'
    })
    return out_df


def annot_category(input_df: pd.DataFrame,
                   master_df: pd.DataFrame):
    """
    カテゴリ ID をひも付けます.

    Args:
        input_df:
            変換するデータ.
            `value_1`  or `JAN` を column として持っている必要があります.
        master_df:
            商品マスタのデータフレーム

    Returns:

    """
    input_df = input_df.rename(columns={'value_1': 'JAN'})
    out_df = pd.merge(input_df['JAN'],
                      master_df[['JAN', 'category_id']], on='JAN', how='left')
    return out_df['category_id']


def make_pay_df(log_df):
    log_df = only_payment_session_record(log_df)
    pay_df = create_payment(log_df)
    pay_df['category_id'] = annot_category(pay_df, master_df=product_master_df)

    # null の category を削除. JAN が紐付かない時に発生する.
    idx_null = pay_df['category_id'].isnull()
    pay_df = pay_df[~idx_null].reset_index(drop=True)
    # Nullが混じっている時 float になるため int へ明示的に戻す
    pay_df['category_id'] = pay_df['category_id'].astype(int)
    return pay_df

## アソシエーション分析

In [16]:
pay_df = make_pay_df(log_df)
print(pay_df["category_id"].nunique())

677


セッション（basket）ごとに、どのカテゴリの商品を購入したかのピボットテーブルを作成
- 横持ちで買った/買っていないを1/0で表したテーブルが必要

In [18]:
basket_df = pd.pivot_table(
    data=pay_df,
    index="session_id",
    columns="category_id",
    values="n_items",
    aggfunc=np.sum,
    fill_value=0    
    ).apply(lambda x: x>0)

In [19]:
basket_df.head()

category_id,0,1,2,3,4,5,7,8,9,11,12,13,14,23,24,26,27,28,29,30,31,32,33,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,71,72,73,74,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,92,93,94,95,96,97,98,99,108,109,110,111,112,113,114,115,116,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,141,142,143,144,145,146,147,148,149,150,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,181,182,183,184,185,186,188,189,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,242,243,244,245,246,247,248,250,251,252,253,254,255,256,257,258,259,260,261,262,263,265,266,267,268,269,270,271,272,273,274,275,276,278,279,280,281,283,284,285,286,287,288,289,290,291,292,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,319,321,322,323,324,325,326,328,329,330,331,332,333,334,335,336,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,405,406,407,408,409,410,411,413,414,415,416,417,418,419,420,421,422,423,424,425,426,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,484,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,502,503,504,505,506,507,508,509,510,511,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,571,572,573,574,575,576,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,594,595,596,597,598,599,600,601,602,604,606,607,608,610,612,613,615,616,617,618,619,620,621,623,624,625,627,628,629,630,631,632,633,634,635,636,639,640,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,693,694,695,696,697,698,699,700,701,702,703,704,705,707,708,709,710,711,712,713,714,715,716,717,718,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,751,752,753,754,755,756,758,759,761,762,763,764,765,766,767,768,769,770,771
session_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,

# support値
購入確率を求める。商品の組み合わせが多いため、確率が一定以上のものに限定する。
- `min_suport`で指定する

In [20]:
freq_items1 = apriori(basket_df, min_support=0.03, use_colnames=True)
freq_items1.sort_values("support", ascending=False).head()

,support,itemsets
92,0.223623,(724)
51,0.214051,(509)
5,0.185729,(114)
35,0.171469,(370)
9,0.160179,(143)


idでなくカテゴリ名で表示できるよう、ゴニョゴニョと・・

In [46]:
# id だけを取り出せるよう str.extract 関数を仕様
freq_items1["item_id"] = freq_items1["itemsets"].astype(str).str.extract('frozenset\({(.+)}\)', expand=False)

In [73]:
# 複数紐づいているものは、別カラムにする
tmp = freq_items1["item_id"].str.split(", ", expand=True)
tmp.rename(columns={0: 'id_1', 1: 'id_2', 2:'id_3'}, inplace=True)

In [74]:
tmp.head()

,id_1,id_2,id_3
0,38,None,None
1,40,None,None
2,50,None,None
3,111,None,None
4,113,None,None


In [75]:
# id ⇒ category名
tmp = tmp.applymap(lambda x: id2cat[int(x)] if x is not None else np.nan).rename(columns={0: 'name_1', 1: 'name_2', 2:'name_3'})

freq_items1_ = pd.concat([freq_items1, tmp], axis=1)

support は itemsets を購入する確率  
例えば、support が一番大きい組み合わせは 野菜__きのこ類 - 野菜__葉茎菜類 である

In [79]:
freq_items1_[~freq_items1_["id_2"].isnull()].sort_values('support', ascending = False).head(10)

,support,itemsets,item_id,id_1,id_2,id_3
199,0.078594,"(716, 724)","716, 724",野菜__きのこ類,野菜__葉茎菜類,NaN
170,0.074662,"(370, 724)","370, 724",和日配__豆腐,野菜__葉茎菜類,NaN
180,0.073188,"(724, 509)","724, 509",野菜__葉茎菜類,日配飲料__牛乳,NaN
204,0.068862,"(720, 724)","720, 724",野菜__土物類,野菜__葉茎菜類,NaN
190,0.066867,"(587, 724)","587, 724",玉子__玉子,野菜__葉茎菜類,NaN
205,0.066170,"(722, 724)","722, 724",野菜__果菜類,野菜__葉茎菜類,NaN
124,0.061727,"(509, 143)","509, 143",日配飲料__牛乳,デザート__ヨーグルト,NaN
174,0.061528,"(587, 509)","587, 509",玉子__玉子,日配飲料__牛乳,NaN
162,0.058535,"(370, 509)","370, 509",和日配__豆腐,日配飲料__牛乳,NaN
152,0.057705,"(368, 370)","368, 370",和日配__納豆,和日配__豆腐,NaN


# Lift値
同時購入の傾向の強さ（単独購入と比べて、合わせ買いの傾向がどれだけ強いか）

In [80]:
# アソシエーションルールの抽出
a_rules1 = association_rules(freq_items1,metric="lift", min_threshold=1)

# リフト値でsort
a_rules1 = a_rules1.sort_values("lift", ascending=False).reset_index(drop=True)

a_rules1.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(370, 724)",(716),0.074662,0.133803,0.034925,0.467778,3.496030,0.024935,1.627510
1,(716),"(370, 724)",0.133803,0.074662,0.034925,0.261021,3.496030,0.024935,1.252185
2,(723),(720),0.081168,0.137080,0.038689,0.476649,3.477150,0.027562,1.648835
3,(720),(723),0.137080,0.081168,0.038689,0.282233,3.477150,0.027562,1.280125
4,(722),(718),0.125126,0.077638,0.030318,0.242297,3.120872,0.020603,1.217314


- antecedent support は antecedents が購入される確率  𝑃(antecedent) 。
- consequents support は consequents が購入される確率  𝑃(consequents) 。
- support は antecedent と consequents が同時に購入される確率  𝑃(antecedent,consequents) 。
- confidence は antecedent が購入された中で consequents が購入される確率  𝑃(antecedent,consequents)/𝑃(antecedent) 。
- lift は antecedent が購入された中で consequents が購入される確率とconsequentsが購入される確率の比  𝑃(antecedent,consequents)/(𝑃(antecedent)∗𝑃(consequents)) 

In [89]:
# id だけを取り出せるよう str.extract 関数を仕様
a_rules1["antecedents_id"] = a_rules1["antecedents"].astype(str).str.extract('frozenset\({(.+)}\)', expand=False)
a_rules1["consequents_id"] = a_rules1["consequents"].astype(str).str.extract('frozenset\({(.+)}\)', expand=False)

# 複数紐づいているものは、別カラムにする
tmp_ante = a_rules1["antecedents_id"].str.split(", ", expand=True)
tmp.rename(columns={0: 'antecedents_id_1', 1: 'antecedents_id_2'}, inplace=True)
tmp_cons = a_rules1["consequents_id"].str.split(", ", expand=True)
tmp.rename(columns={0: 'consequents_id_1', 1: 'consequents_id_2'}, inplace=True)

tmp = pd.concat([tmp_ante, tmp_cons], axis=1)

# id ⇒ category名
tmp = tmp.applymap(lambda x: id2cat[int(x)] if x is not None else np.nan)
tmp.columns = ["antecedents_name_1", "antecedents_name_2", "consequents_name_1", "consequents_name_2"]

In [92]:
a_rules1_ = pd.concat([tmp, a_rules1["lift"]], axis=1).sort_values("lift", ascending=False).reset_index(drop=True)

a_rules1_.head(10)

,antecedents_name_1,antecedents_name_2,consequents_name_1,consequents_name_2,lift
0,和日配__豆腐,野菜__葉茎菜類,野菜__きのこ類,NaN,3.496030
1,野菜__きのこ類,NaN,和日配__豆腐,野菜__葉茎菜類,3.496030
2,野菜__根菜類,NaN,野菜__土物類,NaN,3.477150
3,野菜__土物類,NaN,野菜__根菜類,NaN,3.477150
4,野菜__果菜類,NaN,野菜__トマト,NaN,3.120872
5,野菜__トマト,NaN,野菜__果菜類,NaN,3.120872
6,野菜__葉茎菜類,NaN,和日配__豆腐,野菜__きのこ類,3.058039
7,和日配__豆腐,野菜__きのこ類,野菜__葉茎菜類,NaN,3.058039
8,野菜__根菜類,NaN,野菜__きのこ類,NaN,2.821501
9,野菜__きのこ類,NaN,野菜__根菜類,NaN,2.821501


おもしろい
- スナック・キャンディー_スナックやチョコ・ビスクラ_チョコレートといった菓子は他の菓子、食料品と一緒に購入する傾向にある
- 麺類_カップ麺とスナック・キャンディー_スナックは一緒に購入される傾向が強い。体に良くない。。。

In [98]:
a_rules1_.query('(consequents_name_1 in @TARGET_CATEGORIES) or (consequents_name_2 in @TARGET_CATEGORIES)').head(20)

,antecedents_name_1,antecedents_name_2,consequents_name_1,consequents_name_2,lift
17,チョコ・ビスクラ__ビスクラ,NaN,チョコ・ビスクラ__チョコレート,NaN,2.592324
32,スナック・キャンディー__スナック,NaN,和菓子__米菓,NaN,2.276159
33,和菓子__米菓,NaN,スナック・キャンディー__スナック,NaN,2.276159
42,チョコ・ビスクラ__子供,NaN,スナック・キャンディー__スナック,NaN,2.145927
46,チョコ・ビスクラ__チョコレート,NaN,スナック・キャンディー__スナック,NaN,2.129839
47,スナック・キャンディー__スナック,NaN,チョコ・ビスクラ__チョコレート,NaN,2.129839
49,チョコ・ビスクラ__ビスクラ,NaN,スナック・キャンディー__スナック,NaN,2.113212
170,スナック・キャンディー__スナック,NaN,麺類__カップ麺,NaN,1.541037
171,麺類__カップ麺,NaN,スナック・キャンディー__スナック,NaN,1.541037
187,日配飲料__果汁・野菜,NaN,スナック・キャンディー__スナック,NaN,1.363679


In [95]:
TARGET_CATEGORIES

['ビール系__RTD',
 'ビール系__ビール系',
 'ビール系__ノンアルコール',
 'スナック・キャンディー__スナック',
 'チョコ・ビスクラ__チョコレート',
 'スナック・キャンディー__ガム',
 'スナック・キャンディー__シリアル',
 'アイスクリーム__ノベルティー',
 '和菓子__米菓',
 '水・炭酸水__大型PET（炭酸水）',
 '水・炭酸水__小型PET（炭酸水）',
 '缶飲料__コーヒー（缶）',
 '小型PET__コーヒー（小型PET）',
 '大型PET__無糖茶（大型PET）',
 '麺類__カップ麺']